# Homework 5

Alles Rebel
Computational Science PhD



## Intoduction!



## Pre-Reqs

Like other homeworks, we'll start by getting the kernel set up with the right dependencies. Running this once should do the trick for the rest of the homework;

In [1]:
import Pkg; 
Pkg.add("Distributions")
Pkg.add("PrettyTables")
Pkg.add("Plots")
Pkg.add("ForwardDiff")
Pkg.add("Optim")

using Printf
using LinearAlgebra
using ForwardDiff
using Random
using Distributions
using Statistics
using Dates
using PrettyTables
using Optim
using Plots

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
Precompiling project...
   3163.9 ms  ✓ Plots → IJuliaExt
  1 dependency successfully precompiled in 5 seconds. 240 already precompiled.
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


## Problem 1

The goal of this assignment is to apply a gradient solver to a constrainted optimization problem. In this case the problem needs to be atleast 10 dimensions. The method I'll stick with is the first one suggested by the assignment: the method of Augmented Lagrangian! And we'll apply it to Booth's Function, just updated to use 10 variables instead of 2 via the chaining method introduced in HW2, 1C.

### Broad Overview of Augmented Lagrangians
In general, these methods solve optimization problems with constraints by incorporating the constraints as additional terms in the objective function. These additional terms involve new parameters, known as Lagrange multipliers, which help reformulate the constrained problem (involving equalities or inequalities) into an unconstrained one.

To ensure numerical stablity - Augmented lagrangians intoduce a penality term to the discourage constraint violations. The optimization alternates between minimizing the augmented Lagrangian and updating the multipliers and penalty parameters iteratively. That's what we'll be implementing below!

### Booth's Function!

We'll reuse the code I wrote from the earlier homework - line for line. Originally designed to be used with existing autodiff (forwardiff in julia) and Optim package. It will suit our purposes for this homework. 

In [4]:
# Booth's function - generalized for any number of variables
# Uses the method suggested by Homework2 Problem 1C to chain variables
# pair wise to extend from 2D -> ND
# I wrote this for HW2
function booths_n(x; n = 10)
    result = 0.0
    # same rationale for ending at n-1, we need the nth element for n-1
    for i in 1:(n - 1)
        result += ((x[i] + 2*x[i+1] - 7)^2 + (2*x[i] + x[i+1] - 5)^2)
    end
    return result
end

# Same with this - since the methods of HW needed gradients
# we'll need this as well for this HW
function booths_n_gradient(x; n = 10)
    grad = Float64[]
    
    # Special case for the first element
    push!( grad, 2*(x[1]+2*x[2]-7) + 4*(2*x[1] + x[2] - 5) )
    
    # Compute the gradient for the 2:n-1 elements
    for i in 2:(n-1)
        push!( grad, 2*(x[i]+2*x[i+1]-7) + 4*(2*x[i] + x[i+1] - 5) + 4*(x[i-1] + 2*x[i] - 7) + 2*(2*x[i-1] + x[i] - 5) )
    end

    # Special case for the last element (n-th element)
    push!( grad, 4*(x[n-1] + 2*x[n] - 7) + 2*(2*x[n-1] + x[n] - 5) ) 
    
    return grad
end
    

booths_n_gradient (generic function with 1 method)

### Implemention of augmented lagrangian method

This will be a bit involved, but in general - it's the following steps:
- We'll attempt to use Julia's Optim library
- We'll use the earlier code to compare the results of the augmented lagrangian with


In [5]:
function gradient_descent(objective, x0; tol=1e-6, max_iters=1000)
    """
    Gradient descent implementation using Julia's Optim library.
    
    
    Parameters:
        objective : Objective function to minimize.
        x0        : Initial guess for optimization variables.
        tol       : Tolerance for convergence (default: 1e-6).
        max_iters : Maximum number of iterations (default: 1000).
        
    Returns:
        x_optimal       : Optimized variables.
        steps           : Number of iterations performed.
        func_evals      : Total number of function evaluations.
        grad_evals      : Total number of gradient evaluations.
    """
    result = Optim.optimize(objective, x0, GradientDescent(); 
                      iterations = max_iters, 
                      g_tol = tol)
    x_optimal = Optim.minimizer(result)
    steps = Optim.iterations(result)
    func_evals = Optim.f_calls(result)  # Number of function evaluations
    grad_evals = Optim.g_calls(result)  # Number of gradient evaluations
    return x_optimal, steps, func_evals, grad_evals
end

function augmented_lagrange_method(f, h, x, k_max; ρ=1.0, γ=2.0)
    """
    Augmented Lagrange Method for constrained optimization.
    
    Minimizes `f` subject to `h(x) = 0`.
    
    Parameters:
        f       : Objective function to minimize.
        h       : Constraint function such that `h(x) = 0`.
        x       : Initial guess for optimization variables.
        k_max   : Maximum number of iterations.
        ρ       : Initial penalty parameter (default: 1.0).
        γ       : Penalty parameter scaling factor (default: 2.0).
        
    Returns:
        x                     : Point where minimum occurs.
        descents              : Total number of gradient descent steps performed.
        convergence_measure   : Absolute difference between the last two 
                                minimum values of the objective function.
    """
    λ = zeros(length(h(x)))               # Initialize Lagrange multipliers
    descents = 0                          # Track total descent steps
    old_f_value = f(x)                    # Track old objective value
    convergence_measure = 0.0             # Convergence measure

    for k in 1:k_max
        # Define penalty function
        penalty_function(x) = (ρ / 2) * sum(h(x).^2) - dot(λ, h(x))
        
        # Define augmented objective function
        objective(x) = f(x) + penalty_function(x)
        
        # Optimize using steepest gradient descent
        x, descent_steps = gradient_descent(objective, x)
        
        # Update convergence metrics
        new_f_value = f(x)
        convergence_measure = abs(new_f_value - old_f_value)
        old_f_value = new_f_value

        # Update Lagrange multipliers and penalty parameter
        λ .= λ .- ρ * h(x)
        ρ *= γ

        # Accumulate descent steps
        descents += descent_steps
    end

    return x, descents, convergence_measure
end


augmented_lagrange_method (generic function with 1 method)

### Experimental Methodology

We'll utilize the same testing methodology as the HW2, generating a BUNCH of random vectors to start from. And then running the method across each of the vectors, recording how many function calls, gradient calls, exectuation time and error from what the ideal solution would have been. We'll compare the above method against the earlier line search method.

First we'll get the environment ready to do the experiment - how we'll generate everything an calculate the answers. 